In [8]:
import os
import numpy as np
from PIL import Image

In [8]:
"abeinasdfasbijndfbidnbdinae".find("bin")

-1

In [7]:
a = np.array([[[1,2,3],[4,5,6],[7,8,9]]])
b = np.vstack([a,a,a])
b

array([[[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]])

In [10]:
tot = 0
num = 0
for file in os.listdir("train_dataset/train_lr/"):
    img = Image.open("train_dataset/train_lr/"+file)
    img = np.array(img,dtype=np.float32)
    if 256 <= np.max(img) <= 65535:
        img //= 256.
    if 1 < np.max(img) <= 255:
        img /= 255.
    tot += np.mean(img)
    num += 1
tot/num

0.32344064513842263

In [12]:
tot = 0
num = 0
for file in os.listdir("train_dataset/train_hr/"):
    img = Image.open("train_dataset/train_hr/"+file)
    img = np.array(img,dtype=np.float32)
    if 256 <= np.max(img) <= 65535:
        img //= 256.
    if 1 < np.max(img) <= 255:
        img /= 255.
    tot += np.mean(img)
    num += 1
tot/num

0.35000471224387486

In [13]:
import torch 
import torch.nn as nn

In [118]:
class MeanShift(nn.Conv2d):
    def __init__(self, rgb_range, rgb_mean, rgb_std, sign=-1):
        super(MeanShift, self).__init__(3, 3, kernel_size=1)
        std = torch.Tensor(rgb_std)
        self.weight.data = torch.eye(3).view(3, 3, 1, 1)
        self.weight.data.div_(std.view(3, 1, 1, 1))
        self.bias.data = sign * rgb_range * torch.Tensor(rgb_mean)
        self.bias.data.div_(std)
        self.requires_grad = False

class MeanShiftGrey(nn.Conv2d):
    def __init__(self, rgb_range, rgb_mean, rgb_std, sign=-1):
        super(MeanShiftGrey, self).__init__(1, 1, kernel_size=1)
        std = torch.tensor([rgb_std])
        self.weight.data = torch.eye(1).view(1, 1, 1, 1)
        self.weight.data.div_(std.view(1, 1, 1, 1))
        self.bias.data = sign * rgb_range * torch.tensor([rgb_mean])
        self.bias.data.div_(std)
        self.requires_grad = False
        print(self.bias.shape)

In [139]:
class Some:
    def __init__(self):
        rgb_mean = (0.4488, 0.4371, 0.4040)
        rgb_std = (1.0, 1.0, 1.0)
        self.sub_mean1 = MeanShift(255, rgb_mean, rgb_std)
        self.add_mean1 = MeanShift(255, rgb_mean, rgb_std, 1)
        self.sub_mean2 = MeanShiftGrey(255, (0.4488), (1.0))
        self.add_mean2 = MeanShiftGrey(255, (0.4488), (1.0), 1)

    def forward(self, x):
        x = self.sub_mean1(x)
        y = self.add_mean1(x)
        return x,y
    
    def forward2(self, x):
        x = self.sub_mean2(x)
        y = self.add_mean2(x)
        return x,y

In [140]:
arr = np.random.randint(0,255,[2,3,5,5],dtype=np.int32)

In [141]:
something = Some()
x = torch.tensor(arr,dtype=torch.float32)
y,z = something.forward(x)

torch.Size([1])
torch.Size([1])


In [142]:
other = Some()
c = torch.tensor(arr[:,0].reshape(2,1,5,5),dtype=torch.float32)
a,b = other.forward2(c)

torch.Size([1])
torch.Size([1])


In [143]:
c

tensor([[[[ 27.,  37.,  23., 251., 135.],
          [148., 249., 227.,  90., 226.],
          [ 51., 122., 195., 125., 173.],
          [104.,   1.,  44., 212., 137.],
          [205., 141., 121., 230., 208.]]],


        [[[164., 110., 207., 236., 169.],
          [  7.,  14.,  36.,  20.,  39.],
          [157.,  15., 195.,  77.,  15.],
          [ 28.,  29.,  37.,  54.,  13.],
          [188.,  82., 112., 114., 143.]]]])

In [144]:
x

tensor([[[[ 27.,  37.,  23., 251., 135.],
          [148., 249., 227.,  90., 226.],
          [ 51., 122., 195., 125., 173.],
          [104.,   1.,  44., 212., 137.],
          [205., 141., 121., 230., 208.]],

         [[212.,  77.,  45.,   9., 230.],
          [ 87., 176.,  42., 246.,   2.],
          [197., 235., 162.,  52.,  13.],
          [ 94., 242.,  47., 127., 172.],
          [211.,  77., 173., 200., 239.]],

         [[249., 202., 243.,  15., 120.],
          [224., 197.,  82., 232., 231.],
          [138., 228., 243., 235., 254.],
          [250., 226.,  51.,  17., 177.],
          [115.,  99., 178., 231., 234.]]],


        [[[164., 110., 207., 236., 169.],
          [  7.,  14.,  36.,  20.,  39.],
          [157.,  15., 195.,  77.,  15.],
          [ 28.,  29.,  37.,  54.,  13.],
          [188.,  82., 112., 114., 143.]],

         [[150., 222., 111., 155.,  43.],
          [116.,   4., 181.,  61.,  11.],
          [ 18.,  20., 194., 207., 145.],
          [ 34., 146., 1

In [145]:
y[0]

tensor([[[ -87.4440,  -77.4440,  -91.4440,  136.5560,   20.5560],
         [  33.5560,  134.5560,  112.5560,  -24.4440,  111.5560],
         [ -63.4440,    7.5560,   80.5560,   10.5560,   58.5560],
         [ -10.4440, -113.4440,  -70.4440,   97.5560,   22.5560],
         [  90.5560,   26.5560,    6.5560,  115.5560,   93.5560]],

        [[ 100.5395,  -34.4605,  -66.4605, -102.4605,  118.5395],
         [ -24.4605,   64.5395,  -69.4605,  134.5395, -109.4605],
         [  85.5395,  123.5395,   50.5395,  -59.4605,  -98.4605],
         [ -17.4605,  130.5395,  -64.4605,   15.5395,   60.5395],
         [  99.5395,  -34.4605,   61.5395,   88.5395,  127.5395]],

        [[ 145.9800,   98.9800,  139.9800,  -88.0200,   16.9800],
         [ 120.9800,   93.9800,  -21.0200,  128.9800,  127.9800],
         [  34.9800,  124.9800,  139.9800,  131.9800,  150.9800],
         [ 146.9800,  122.9800,  -52.0200,  -86.0200,   73.9800],
         [  11.9800,   -4.0200,   74.9800,  127.9800,  130.9800]]],
    

In [146]:
a

tensor([[[[ -87.4440,  -77.4440,  -91.4440,  136.5560,   20.5560],
          [  33.5560,  134.5560,  112.5560,  -24.4440,  111.5560],
          [ -63.4440,    7.5560,   80.5560,   10.5560,   58.5560],
          [ -10.4440, -113.4440,  -70.4440,   97.5560,   22.5560],
          [  90.5560,   26.5560,    6.5560,  115.5560,   93.5560]]],


        [[[  49.5560,   -4.4440,   92.5560,  121.5560,   54.5560],
          [-107.4440, -100.4440,  -78.4440,  -94.4440,  -75.4440],
          [  42.5560,  -99.4440,   80.5560,  -37.4440,  -99.4440],
          [ -86.4440,  -85.4440,  -77.4440,  -60.4440, -101.4440],
          [  73.5560,  -32.4440,   -2.4440,   -0.4440,   28.5560]]]],
       grad_fn=<ConvolutionBackward0>)

In [53]:
z[0]

tensor([[[ 91., 253., 173., 252., 147.],
         [172.,  80., 200.,  39.,  98.],
         [215., 162., 182.,  87., 210.],
         [113., 193., 176.,  88., 182.],
         [  7.,  74.,   4.,  80., 217.]],

        [[192.,  19., 146., 238., 150.],
         [238.,  53., 233., 114., 211.],
         [228., 251.,  28., 232., 140.],
         [ 14.,  81., 240.,  48., 143.],
         [ 24., 251., 224.,   9., 128.]],

        [[116.,  58.,   2., 225., 254.],
         [ 60.,  24., 215., 239., 201.],
         [  0., 178.,  59., 156., 188.],
         [224., 145., 195., 192., 142.],
         [196.,  31., 254.,  11.,  63.]]], grad_fn=<SelectBackward0>)

In [50]:
z[1]

tensor([[[187., 219.,  49., 140., 109.],
         [170., 160.,  16.,  21., 200.],
         [176., 247., 238.,  16., 253.],
         [140.,  32., 203.,  23., 181.],
         [165., 214., 160., 123., 116.]],

        [[ 48., 241., 139., 111., 196.],
         [ 66.,  65., 254., 246.,  82.],
         [142.,   0.,  36., 202., 248.],
         [210.,  45., 167.,  78.,  41.],
         [ 91., 191., 132.,   7.,  79.]],

        [[163., 206.,  92., 102., 165.],
         [  9., 164., 176., 197.,  75.],
         [  7.,  83.,  76.,  23., 141.],
         [216., 192.,  70., 112., 121.],
         [ 16.,  47.,  27.,  26.,  52.]]], grad_fn=<SelectBackward0>)

In [54]:
x.shape

torch.Size([2, 3, 5, 5])

In [55]:
z[0].shape

torch.Size([3, 5, 5])

In [152]:
torch.tensor([1,2.1,3.8]).cuda().device

device(type='cuda', index=0)

In [ ]:
import scipy.misc as misc
from imageio import imsave